# NISQ Experiment

In [3]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from quantum_circuits import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import random_unitary, Operator
from qiskit.providers.fake_provider import FakeManilaV2

from qiskit import Aer

In [5]:
#qk.IBMQ.save_account("aaff224a30e77d3ea3af33e2e3fb1cebfbff4e33b04ef75fdbbb9e9055426f547739b68d5b93dac4d47e7828d840a212436b1c36a358f9185574ce60af9a9f53", overwrite=True) 
provider = qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')

## Ring Disk Transition, Three Qubits

### 8 Layers

In [ ]:
backend = provider.get_backend("ibmq_belem")

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

for i in range(10):
    circuit_target = pqc_basic(n, 8).reverse_bits()
    circuit_target_list.append(circuit_target)

In [10]:
belem_3qubits_8layers = ExecuteAndCollect()
belem_3qubits_8layers.setup_circuits(circuit_target_list, N_map = 2000-6**n)

belem_3qubits_8layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_3qubits_8layers", 
                                        concatenate=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 16 Layers

In [5]:
backend = provider.get_backend("ibmq_belem")

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

for i in range(10):
    circuit_target = pqc_basic(n, 16).reverse_bits()
    circuit_target_list.append(circuit_target)

In [6]:
belem_3qubits_16layers = ExecuteAndCollect()
belem_3qubits_16layers.setup_circuits(circuit_target_list, N_map = 2000-6**n)

belem_3qubits_16layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_3qubits_16layers", 
                                        concatenate=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 24 Layers

In [7]:
backend = provider.get_backend("ibmq_belem")

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

for i in range(10):
    circuit_target = pqc_basic(n, 24).reverse_bits()
    circuit_target_list.append(circuit_target)

In [8]:
belem_3qubits_24layers = ExecuteAndCollect()
belem_3qubits_24layers.setup_circuits(circuit_target_list, N_map = 2000-6**n)

belem_3qubits_24layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_3qubits_24layers", 
                                        concatenate=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 32 Layers

In [5]:
backend = provider.get_backend("ibmq_belem")

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

for i in range(10):
    circuit_target = pqc_basic(n, 32).reverse_bits()
    circuit_target_list.append(circuit_target)

In [6]:
belem_3qubits_32layers = ExecuteAndCollect()
belem_3qubits_32layers.setup_circuits(circuit_target_list, N_map = 2000-6**n)

belem_3qubits_32layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_3qubits_32layers", 
                                        concatenate=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 40 Layers

In [7]:
backend = provider.get_backend("ibmq_belem")

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

for i in range(10):
    circuit_target = pqc_basic(n, 40).reverse_bits()
    circuit_target_list.append(circuit_target)

In [8]:
belem_3qubits_40layers = ExecuteAndCollect()
belem_3qubits_40layers.setup_circuits(circuit_target_list, N_map = 2000-6**n)

belem_3qubits_40layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_3qubits_40layers", 
                                        concatenate=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 48 Layers

In [5]:
backend = provider.get_backend("ibmq_belem")

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

for i in range(10):
    circuit_target = pqc_basic(n, 48).reverse_bits()
    circuit_target_list.append(circuit_target)

In [6]:
belem_3qubits_48layers = ExecuteAndCollect()
belem_3qubits_48layers.setup_circuits(circuit_target_list, N_map = 2000-6**n)

belem_3qubits_48layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_3qubits_48layers", 
                                        concatenate=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

### 56 Layers

In [5]:
backend = provider.get_backend("ibmq_belem")

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

for i in range(10):
    circuit_target = pqc_basic(n, 56).reverse_bits()
    circuit_target_list.append(circuit_target)

In [6]:
belem_3qubits_56layers = ExecuteAndCollect()
belem_3qubits_56layers.setup_circuits(circuit_target_list, N_map = 2000-6**n)

belem_3qubits_56layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_3qubits_56layers", 
                                        concatenate=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Concatenate Circuits

In [11]:
backend = provider.get_backend("ibmq_lima")

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target1 = pqc_basic(n, 8).reverse_bits()
circuit_target2 = pqc_basic(n, 8).reverse_bits()

circuit_target12 = deepcopy(circuit_target1)
circuit_target12 = circuit_target12.compose(circuit_target2)
circuit_target_list = [circuit_target12]

In [12]:
belem_concatenate = ExecuteAndCollect()
belem_concatenate.setup_circuits(circuit_target_list, N_map = 2000-6**n)

belem_concatenate.execute_circuits(backend=backend, 
                                   shots_map = 1024, 
                                   shots_spam = 1024, 
                                   filename="lima_concatenate_total", 
                                   concatenate=True)

  0%|          | 0/4 [00:00<?, ?it/s]

IBMQBackendJobLimitError: "Error submitting job: '400 Client Error: Bad Request for url: https://api.quantum-computing.ibm.com/api/Network/ibm-q/Groups/open/Projects/main/Jobs. Reached maximum number (5) of concurrent jobs, Error code: 3458.'"

## Four Qubits

In [9]:
backend = provider.get_backend("ibmq_belem")

n = 4
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

circuit_target = pqc_basic(n, 8).reverse_bits()
circuit_target_list.append(circuit_target)

In [10]:
belem_4qubits_8layers = ExecuteAndCollect()
belem_4qubits_8layers.setup_circuits(circuit_target_list, N_map = 10000-6**n)

belem_4qubits_8layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_4qubits_8layers", 
                                        concatenate=True)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
backend = provider.get_backend("ibmq_belem")

n = 4
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

circuit_target = pqc_basic(n, 16).reverse_bits()
circuit_target_list.append(circuit_target)

In [7]:
belem_4qubits_16layers = ExecuteAndCollect()
belem_4qubits_16layers.setup_circuits(circuit_target_list, N_map = 10000-6**n)

belem_4qubits_16layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_4qubits_16layers", 
                                        concatenate=True)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
backend = provider.get_backend("ibmq_belem")

n = 4
d = 2**n

np.random.seed(42)
random.seed(42)
circuit_target_list = []

circuit_target = pqc_basic(n, 4).reverse_bits()
circuit_target_list.append(circuit_target)

In [ ]:
belem_4qubits_4layers = ExecuteAndCollect()
belem_4qubits_4layers.setup_circuits(circuit_target_list, N_map = 10000-6**n)

belem_4qubits_4layers.execute_circuits(backend=backend, 
                                        shots_map = 1024, 
                                        shots_spam = 1024, 
                                        filename="belem_4qubits_4layers", 
                                        concatenate=True)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]